In [1]:
import pandas as pd
import os
import glob

Concatenate all shot dfs for each game. Final result should be all shots from the dataset

In [76]:
files = glob.glob("./data/game_shots/*.csv")

df_list = []
for file in files:
    df = pd.read_csv(file)
    df_list.append(df)
    
final_df = pd.concat(df_list)

['./data/game_shots/0021500502_home.csv',
 './data/game_shots/0021500503_home.csv',
 './data/game_shots/0021500527_visitor.csv',
 './data/game_shots/0021500077_visitor.csv',
 './data/game_shots/0021500115_visitor.csv',
 './data/game_shots/0021500498_home.csv',
 './data/game_shots/0021500445_visitor.csv',
 './data/game_shots/0021500499_home.csv',
 './data/game_shots/0021500230_home.csv',
 './data/game_shots/0021500056_home.csv',
 './data/game_shots/0021500057_home.csv',
 './data/game_shots/0021500231_home.csv',
 './data/game_shots/0021500557_visitor.csv',
 './data/game_shots/0021500007_visitor.csv',
 './data/game_shots/0021500198_visitor.csv',
 './data/game_shots/0021500165_visitor.csv',
 './data/game_shots/0021500435_visitor.csv',
 './data/game_shots/0021500591_home.csv',
 './data/game_shots/0021500338_home.csv',
 './data/game_shots/0021500339_home.csv',
 './data/game_shots/0021500372_visitor.csv',
 './data/game_shots/0021500622_visitor.csv',
 './data/game_shots/0021500279_home.csv',
 

In [77]:
#make our game data actually a date, needed so we can do predictions
final_df['GAME_DATE'] = pd.to_datetime(final_df['GAME_DATE'],format = "%Y%m%d")
final_df.sort_values(by=['GAME_DATE'],ascending=True,inplace=True)
print(final_df['GAME_DATE'].head())
print(final_df['GAME_DATE'].tail())
print(len(final_df))
final_df.to_csv("./data/all_shots_raw.csv")

In [83]:
#merge with the NBA Savant data
shot_clock_info1 = pd.read_csv("./data/nba_savant.csv")
shot_clock_info2 = pd.read_csv("./data/nba_savant (1).csv")
shot_clock_info3 = pd.read_csv("./data/nba_savant (2).csv")
shot_clock_info = pd.concat([shot_clock_info1,shot_clock_info2,shot_clock_info3])
print(len(shot_clock_info))
shot_clock_info.head()

111084


,name,team_name,game_date,season,espn_player_id,team_id,espn_game_id,period,minutes_remaining,seconds_remaining,...,shot_type,shot_distance,opponent,x,y,dribbles,touch_time,defender_name,defender_distance,shot_clock
0,Jimmy Butler,Chicago Bulls,2015-10-27,2015,6430.0,1610612741,400827889,1,8,14,...,2PT Field Goal,0,Cleveland Cavaliers,0,1,0,0.0,"Mozgov, Timofey",3.2,13.8
1,John Henson,Milwaukee Bucks,2015-11-30,2015,6592.0,1610612749,400828145,4,5,43,...,2PT Field Goal,0,Denver Nuggets,0,1,0,0.0,"Jokic, Nikola",4.0,9.7
2,Jeremy Evans,Dallas Mavericks,2015-10-28,2015,4295.0,1610612742,400827901,4,11,50,...,2PT Field Goal,2,Phoenix Suns,21,8,0,1.1,"Warren, TJ",3.1,15.0
3,AlFarouq Aminu,Portland Trail Blazers,2015-11-30,2015,4248.0,1610612757,400828148,2,1,34,...,2PT Field Goal,0,Los Angeles Clippers,0,1,0,0.0,"Jordan, DeAndre",7.0,10.7
4,Steven Adams,Oklahoma City Thunder,2015-10-28,2015,2991235.0,1610612760,400827900,4,8,44,...,2PT Field Goal,2,San Antonio Spurs,18,15,0,0.0,"Diaw, Boris",2.2,10.8


In [84]:
# also convert to date so we can join with raw shot data
shot_clock_info['game_date'] = pd.to_datetime(shot_clock_info['game_date'])
#since game ids are not the same, merge shot data on the info below. does not appear to be an issue
final_df = final_df.merge(shot_clock_info,left_on=['TEAM_NAME','PERIOD','MINUTES_REMAINING','SECONDS_REMAINING','GAME_DATE'],right_on=['team_name','period','minutes_remaining','seconds_remaining','game_date'])
#save to csv
final_df.to_csv("./data/all_shots.csv")

In [86]:
#just seeing whats null, potential issues with sportvu data
final_df[['shooter_x', 'shooter_y', 'home_id',
       '1st_closest_defender_dist', '1st_closest_defender_angle',
       '2nd_closest_defender_dist', '2nd_closest_defender_angle',
       '3rd_closest_defender_dist', '3rd_closest_defender_angle', 'name',
       'team_name', 'game_date', 'season', 'espn_player_id', 'team_id',
       'espn_game_id', 'period', 'minutes_remaining', 'seconds_remaining',
       'shot_made_flag', 'action_type', 'shot_type', 'shot_distance',
       'opponent', 'x', 'y', 'dribbles', 'touch_time', 'defender_name',
       'defender_distance', 'shot_clock']].isnull().sum()

shooter_x                      255
shooter_y                      255
home_id                          0
1st_closest_defender_dist      255
1st_closest_defender_angle     605
2nd_closest_defender_dist      255
2nd_closest_defender_angle     255
3rd_closest_defender_dist      255
3rd_closest_defender_angle     255
name                             0
team_name                        0
game_date                        0
season                           0
espn_player_id                1928
team_id                          0
espn_game_id                     0
period                           0
minutes_remaining                0
seconds_remaining                0
shot_made_flag                   0
action_type                      0
shot_type                        0
shot_distance                    0
opponent                         0
x                                0
y                                0
dribbles                         0
touch_time                       0
defender_name       